In [1]:
using Pkg
Pkg.activate("/media/mat/HDD/AdaptiveTransportMap/")

 Activating environment at `/media/mat/HDD/AdaptiveTransportMap/Project.toml`


In [2]:
using Revise
using AdaptiveTransportMap
using LinearAlgebra
using Test
using ForwardDiff
using SpecialFunctions
using Polynomials
using BenchmarkTools
using TransportMap
using QuadGK
using FastGaussQuadrature
using AdaptiveTransportMap: derivative, vander, scale!, evaluate
using DiffResults
using Distributions
using Random
using LoopVectorization
using Optim

### Greedy fit

In [23]:
k = 2
Ne = 8
m = 5

X  =  Matrix([0.267333   1.43021;
          0.364979   0.607224;
         -1.23693    0.249277;
         -2.0526     0.915629;
         -0.182465   0.415874;
          0.412907   1.01672;
          1.41332   -0.918205;
          0.766647  -1.00445]');

H = HermiteMapk(m, k; α = 0.0)
S = Storage(H.I.f, X);

J = 0.0
dJ = zeros(1)
J = negative_log_likelihood!(J, dJ, getcoeff(H), S, H, X)

(i, dJ) = (1, [0.0])
(i, dJ) = (2, [0.0])
(i, dJ) = (3, [0.0])
(i, dJ) = (4, [0.0])
(i, dJ) = (5, [0.0])
(i, dJ) = (6, [0.0])
(i, dJ) = (7, [0.0])
(i, dJ) = (8, [0.0])


1.3172777885451104

In [22]:
@show J, dJ

(J, dJ) = (1.3172777885451104, [-0.12555625])


(1.3172777885451104, [-0.12555625])

In [33]:
Nx = 2
Ne = 8
m = 5
ens = EnsembleState(Nx, Ne)

ens.S .=  [0.267333   1.43021;
          0.364979   0.607224;
         -1.23693    0.249277;
         -2.0526     0.915629;
         -0.182465   0.415874;
          0.412907   1.01672;
          1.41332   -0.918205;
          0.766647  -1.00445]';
X = ens.S

coeff = [0.0];

# coeff = [0.6285037650645056;
#           -0.4744029092496623;
#            1.1405280011620331;
#           -0.7217760771894809;
#            0.11855056306742319];
# Initialize map with zero coefficients
# Hk_old = HermiteMapk(m, Nx, [0 0; 0 1; 0 2; 1 0; 2 0], coeff; α = 1e-2);
Hk_old = HermiteMapk(m, Nx, [0 0], coeff; α = 1e-2);
S = Storage(Hk_old.I.f, X)

# reduced_margin = getreducedmargin(getidx(Hk_old))
# @show reduced_margin
# @time idx_new, reduced_margin = update_component(Hk_old, X, reduced_margin, S)

# @show idx_new
# @show reduced_margin

J = 0.0
dJ = zeros(1)
J = negative_log_likelihood!(J, dJ, getcoeff(Hk_old), S, Hk_old, X)

@show J, dJ
# idx_new = vcat(getidx(Hk_old), reduced_margin)
# Hk_new = HermiteMapk(m, Nx, idx_new, vcat(getcoeff(Hk_old), zeros(size(reduced_margin,1))); α = 1.0);
# # coeff_new, coeff_idx_added, idx_added = update_coeffs(Hk_old, Hk_new)

(J, dJ) = (1.3172777885451104, [0.33903487500000007])


In [34]:
res = Optim.optimize(Optim.only_fg!(negative_log_likelihood!(S, Hk_old, ens.S)), coeff, Optim.BFGS())
coeffopt = Optim.minimizer(res)

1-element Array{Float64,1}:
 -0.33238713235294115

In [35]:
?break

search: break AbstractVecOrMat benchmarkparallel_TMap_lorenz96



```
break
```

Break out of a loop immediately.

# Examples

```jldoctest
julia> i = 0
0

julia> while true
           global i += 1
           i > 5 && break
           println(i)
       end
1
2
3
4
5
```
